# Задача 1.1

Для решения этой задачи рассмотрим граф, в котором вершины соответствуют людям, а ребра — передачам новости. Начальная вершина помечена как $v_0$. Нам нужно найти вероятность того, что новость не вернется к вершине $v_0$ в течение $t$ шагов.

Заметим, что на каждом шаге новость может перейти только на одного из $n$ соседей текущей вершины. Таким образом, каждый узел графа имеет степень $n$, а всего вершин $n+1$.
Сколькими всего способами могла двигаться новость? Давайте поймем, что если она пропала через t шагов, то у нее есть столько вариантов:
$$ C(t) = \frac{n!}{(n - t)!} * n! $$
Очевидно, что через n + 1 шаг новость точно не будет никем повторена.
Давайте поймем, что если новость вернулась к первому человеку за $t$ шагов, значит что у нас сформировался один простой цикл из $t$ вершин в графе.
Таким образом можно посчитать количество вариантов, которыми новость могла вернуться за t шагов
$$ C_{a}(t) = \frac{n!}{(n - t + 1)! (t - 1)!} (t - 1)! = \frac{n!}{(n - t + 1)!} $$
Тогда можно посчитать вероятность того, что новость за r вернется к $v_0$:
$$ P_{a}(r) = \frac{\sum_{t=0}^r C_{a}(t)}{\sum_{t=0}^n C(t)} = \frac{\sum_{t=0}^r \frac{n!}{(n - t + 1)!}}{n! \sum_{t=0}^n \frac{n!}{(n - t)!}} = \frac{\sum_{t=0}^r \frac{1}{(n - t + 1)!}}{n! \sum_{t=0}^n \frac{1}{(n - t)!}} $$
А для решения пункта б) мы уже посчитали всё что нужно, ведь нам нужно просто узнать, не закончилась ли новость раньше r шагов:
$$ P_{b}(r) = \frac{\sum_{t=0}^{r-1} C(t)}{\sum_{t=0}^n C(t)} = \frac{\sum_{t=0}^{r-1} \frac{1}{(n - t)!}}{\sum_{t=0}^n \frac{1}{(n - t)!}} $$

# Задача 2.2


Перефразируем задачу: у нас есть 3 случайные помеченные точки на отрезке [0, 60], нужно найти вероятность того, что минимальное расстояние между двумя из них <= 10.
Посчитаем меру множества, на котором условие выполнено, по-тупому рассмотрев все положения точек. Для этого посчитаем 3 способа расположить точки:
1) между первой и второй точкой <= 10, а между второй и третьей > 10
$$ \int_{0}^{40} \int_{x}^{x+10} \int_{y + 10}^{60}dz dy dx + \int_{40}^{50} \int_{x}^{50} \int_{y + 10}^{60}dz dy dx = \frac{30500}{3}  $$
2) между второй и третьей точкой <= 10, а между первой и второй > 10
$$ \int_{20}^{60} \int_{z-10}^{z} \int_{0}^{y - 10}dx dy dz + \int_{10}^{20} \int_{10}^{z} \int_{0}^{y - 10}dx dy dz = \frac{30500}{3}  $$
3) между первой и второй точкой <= 10, между второй и третьей <= 10
$$ \int_{0}^{40} \int_{x}^{x+10} \int_{y}^{y+10} dz dy dx + \int_{40}^{50} \int_{x}^{50} \int_{y}^{y + 10} dz dy dx + \int_{40}^{50} \int_{50}^{x + 10} \int_{y}^{60} dz dy dx + \int_{50}^{60} \int_{x}^{60} \int_{y}^{60} dz dy dx = 5000 $$

Сумма этих трех событий получается $ \frac{76000}{3}$. Также это число нужно домножить на $3! = 6$, так как мы рассматривали конкретную расстановку трех точек, а товарищи разные и могли придти в любой из перестановок.
Так общая Лебегова мера пространства равна $60 ^ 3 = 216000$
Тогда искомая вероятность равна $ \frac {2 * 76000}{ 216000} = \frac{19}{27} ~= 0.703704 $
Проверим простым численным моделированием:

In [56]:
import math

import numpy as np

ATTEMPTS = 1000000
success = 0
for _ in range(ATTEMPTS):
    x, y, z = np.random.uniform(0, 60, 3)
    if min(abs(x - y), abs(x - z), abs(y - z)) <= 10:
        success += 1
print(success / ATTEMPTS)

0.703159


Сходится, повезло

# Задача 3.3


Введем событие $C_j = \{X + Y = j\}$
По сути, нам нужно посчитать:
$$ P(A_i | C_j) $$
Воспользуемся формулой Байеса:
$$ P(A_i | C_j) = \frac{P(C_j| A_i) P(A_i)}{P(C_j)} $$
Заметим, что
$$ P(C_j | A_i) = P(B_{j-i}) $$
А также мы можем выразить $C_j$
$$ P(C_j) = \sum_{z=0}_j P(A_z) P(B_{j-z}) = \sum_{z=0}^j p^2 (1-p)^j = (j+1) p^2 (1-p)^j $$
Тогда искомое:
$$ P(A_i | C_j) = \frac{P(b_{j-i}) P(A_i)}{(j+1) p^2 (1-p)^j} = \frac{p^2(1-p)^j}{(j+1) p^2 (1-p)^j} = \frac {1}{j+1}$$

Как ни странно, ответ у нас получился независим от p.
Попробуем проэмулировать эту задачку:

In [ ]:
from collections import Counter
import random
import matplotlib.pyplot as plt
random.seed(42)

MAX = 10000

def simulate(p):
    i = 0
    while True:
        if random.random() < ((1-p) ** i) * p:
            return i
        i += 1
        if i == MAX:
            return i
pt = []
otklt = []
for p in range(35, 85):
    p = p / 100
    num_samples = 5000
    samples_x = list(filter(lambda c: c < MAX, [simulate(p) for i in range(num_samples)]))
    samples_y = list(filter(lambda c: c < MAX, [simulate(p) for i in range(num_samples)]))
    samples_xy = [[x, y] for x in samples_y for y in samples_x]
    c = {i[0] + i[1]: [] for i in samples_xy}
    for i in samples_xy:
        c[i[0] + i[1]].append(i[0])
    sum_otkl = 0
    for j, values in c.items():
        counter = Counter()
        for i in values:
            counter[i] += 1
        sum_otkl += sum(map(lambda it: (it / len(values) - 1 / (j + 1)) ** 2, counter.values()))
    pt.append(p)
    otklt.append(sum_otkl)
plt.plot(pt, otklt)
plt.show()

Как мы видим, среднеквадратичное отклонение от предполагаемого результата получилось относительно небольшим (учитывая, что пар измерений порядка 250000) и при исследованных всех p ведет себя примерно одинаково (я не брал p близкие к 0 или 1, так как на них эмуляция идет очень долго)

# Задача 4

Воспользуемся интегральной теоремой Муавра-Лапласа
$$ P(\frac{n}{2} - \sqrt{npq} <= S_n <= \frac{n}{2} + \sqrt{npq}) =
P(\frac{n}{2} - \sqrt{npq} - np <= S_n - np <= \frac{n}{2} + \sqrt{npq} - np) =
P(\frac{\frac{n}{2} - np}{\sqrt{npq}} - 1 <= \frac{S_n - np}{\sqrt{npq}} <= \frac{\frac{n}{2} - np}{\sqrt{npq}} + 1)
≈ Ф(\frac{\frac{n}{2} - np}{\sqrt{npq}} + 1) - Ф(\frac{\frac{n}{2} - np}{\sqrt{npq}} - 1) $$
Здесь Ф -- функция стандартного нормального распределния. Для её приближения мы воспользуемся функцией из библиотеки scipy (поскольку интеграл из неё всё равно не вычисляется явно).

In [26]:
from fractions import Fraction
from math import ceil, floor, comb
from scipy.stats import norm
print("n\p       0.001      0.01     0.1      0.25     0.5")
for n in (10, 100, 1000, 10000):
    print("{:6}".format(n), end=" ")
    for p in (Fraction(1, 1000), Fraction(1, 100), Fraction(1, 10), Fraction(1, 4), Fraction(1, 2)):
        exact = Fraction(0)
        q = 1 - p
        sqrt_npq = (n * p * q) ** 0.5
        for k in range(ceil(n / 2 - sqrt_npq), floor(n / 2 + sqrt_npq) + 1):
            exact += comb(n, k) * p ** k * q ** (n - k)
        approx = norm.cdf((n / 2 - n * p) / sqrt_npq + 1) - norm.cdf((n / 2 - n * p) / sqrt_npq - 1)
        print("{:2.2e}".format(abs(exact - approx)), end=" ")
    print('')

n\p       0.001      0.01     0.1      0.25     0.5
    10 2.51e-13 2.40e-08 8.39e-04 1.85e-02 2.64e-02 
   100 9.60e-122 6.10e-72 3.61e-21 3.35e-06 4.61e-02 
  1000 0.00e+00 0.00e+00 1.33e-215 1.50e-58 9.63e-03 
 10000 0.00e+00 0.00e+00 0.00e+00 0.00e+00 4.82e-03 


Видно, что при маленьких p интегральная теорема Муавра-Лапласа довольно точно приближала точное значение, но при достаточно больших уже давала значительные огрехи.
С увеличением n, как и ожидается в теории, погрешность уменьшается и стремится к нулю.
